In [1]:
import os  # Import the os module to interact with the operating system


In [2]:
%pwd  # This is a Jupyter Notebook magic command to display the current working directory


'c:\\Users\\ayupt\\Desktop\\Data Science Projects\\End to End Deployment\\Kidney-Disease-Classificaion-End-to-End-MLflow-DVC\\research'

In [3]:
os.chdir("../")  # Change the current working directory to the parent directory


In [4]:
%pwd  # Again, display the updated working directory to confirm the change


'c:\\Users\\ayupt\\Desktop\\Data Science Projects\\End to End Deployment\\Kidney-Disease-Classificaion-End-to-End-MLflow-DVC'

In [5]:
from dataclasses import dataclass  # Importing dataclass to create immutable data structures
from pathlib import Path  # Importing Path to handle file system paths

# Defining a data class to store configuration settings for preparing a base model
@dataclass(frozen=True)  # `frozen=True` makes the data class immutable
class PrepareBaseModelConfig:
    root_dir: Path  # The root directory for the project
    base_model_path: Path  # Path to the pre-trained base model
    updated_base_model_path: Path  # Path where the updated model will be saved
    params_image_size: list  # The size of input images (e.g., [224, 224, 3])
    params_learning_rate: float  # Learning rate for training the model
    params_include_top: bool  # Whether to include the top layer of the base model
    params_weights: str  # Path or identifier for the pre-trained weights (e.g., "imagenet")
    params_classes: int  # Number of output classes for classification



In [6]:
# Importing all constants defined in the constants module
from cnnClassifier.constants import *  

# Importing utility functions for reading YAML files and creating directories
from cnnClassifier.utils.common import read_yaml, create_directories  


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,  # Path to the main config file
        params_filepath=PARAMS_FILE_PATH  # Path to the parameters config file
    ):
        # Read configuration files
        self.config = read_yaml(config_filepath)  # Load main YAML config
        self.params = read_yaml(params_filepath)  # Load parameters YAML config

        # Create necessary directories for storing artifacts
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """
        Retrieves and prepares the configuration for the base model setup.
        Returns an instance of `PrepareBaseModelConfig`.
        """
        config = self.config.prepare_base_model  # Extract base model config section
        
        # Ensure the root directory for preparing the base model exists
        create_directories([config.root_dir])

        # Create and return a PrepareBaseModelConfig instance with required parameters
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),  # Convert to Path object for better handling
            base_model_path=Path(config.base_model_path),  # Path to the pre-trained model
            updated_base_model_path=Path(config.updated_base_model_path),  # Path for saving the updated model
            params_image_size=self.params.IMAGE_SIZE,  # Image size for training
            params_learning_rate=self.params.LEARNING_RATE,  # Learning rate for training
            params_include_top=self.params.INCLUDE_TOP,  # Whether to include the top layer
            params_weights=self.params.WEIGHTS,  # Pre-trained weights to use
            params_classes=self.params.CLASSES  # Number of output classes
        )

        return prepare_base_model_config  # Return the configuration object


In [9]:
import os  # Importing the os module to interact with the operating system (e.g., file paths, directories)

import urllib.request as request  # Importing urllib's request module to handle downloading files from a URL

from zipfile import ZipFile  # Importing ZipFile to work with zip archives (extracting or compressing files)

import tensorflow as tf  # Importing TensorFlow for deep learning model training and inference


In [10]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        """
        Initializes the PrepareBaseModel class with the provided configuration.
        
        Args:
            config (PrepareBaseModelConfig): Configuration object containing model settings.
        """
        self.config = config  # Storing configuration object

    def get_base_model(self):
        """
        Loads the pre-trained VGG16 model with specified configurations and saves it.
        """
        # Load VGG16 model with specified parameters
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,  # Input image dimensions
            weights=self.config.params_weights,  # Pre-trained weights (e.g., "imagenet")
            include_top=self.config.params_include_top  # Whether to include the top layer
        )

        # Save the base model to the specified path
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        """
        Prepares the full model by adding a custom classifier on top of the base model.

        Args:
            model (tf.keras.Model): Base model (e.g., VGG16).
            classes (int): Number of output classes.
            freeze_all (bool): Whether to freeze all layers.
            freeze_till (int or None): Number of layers to keep trainable (if None, all are frozen).
            learning_rate (float): Learning rate for the optimizer.

        Returns:
            full_model (tf.keras.Model): The compiled model ready for training.
        """
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False  # Freeze all layers
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:  # Freeze all layers except the last `freeze_till`
                layer.trainable = False

        # Adding a Flatten layer to convert feature maps into a 1D vector
        flatten_in = tf.keras.layers.Flatten()(model.output)

        # Adding a Dense layer for classification with softmax activation
        prediction = tf.keras.layers.Dense(
            units=classes,  # Number of output classes
            activation="softmax"  # Activation function for multi-class classification
        )(flatten_in)

        # Creating the final model
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        # Compiling the model with SGD optimizer and categorical cross-entropy loss
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]  # Tracking accuracy during training
        )

        full_model.summary()  # Display model summary
        return full_model  # Return the compiled model

    def update_base_model(self):
        """
        Updates the base model by adding a classification head and saving the updated model.
        """
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,  # Number of classes
            freeze_all=True,  # Freezing all base model layers
            freeze_till=None,  # Not unfreezing any layers
            learning_rate=self.config.params_learning_rate  # Learning rate for training
        )

        # Save the updated model
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the given TensorFlow model to the specified path.

        Args:
            path (Path): The path where the model should be saved.
            model (tf.keras.Model): The TensorFlow model to save.
        """
        model.save(path)  # Save the model in HDF5 format


In [ ]:
try:
    # Create an instance of ConfigurationManager to handle configurations
    config = ConfigurationManager()
    
    # Retrieve the configuration settings for preparing the base model
    prepare_base_model_config = config.get_prepare_base_model_config()
    
    # Create an instance of PrepareBaseModel with the retrieved configuration
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    
    # Load and save the pre-trained base model (VGG16 in this case)
    prepare_base_model.get_base_model()
    
    # Modify the base model by adding a classification head and save the updated model
    prepare_base_model.update_base_model()

except Exception as e:
    # If any exception occurs during execution, raise it for debugging
    raise e
